In [1]:
import time
import json
import redis
import asyncio
import numpy as np
from aredis import StrictRedis

In [2]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [33]:
date = time.strftime('%Y%m%d')

check_points_length = int(rd.get(f'hq_{date}_check_points_length'))
assist_count = int(rd.get('hq_assist_count'))
assist_count

7

In [68]:
await ar.pubsub_channels()

[]

In [ ]:
status = np.zeros(check_points_length, dtype=int)
status

In [70]:
def message_handler(message):
    
    assist_idx = str(message['channel']).split('_')[2]
    data = json.loads(message['data'])
    
    if data['status'] == 'successful':
        check_point_idx = int(data['idx'])
        check_point = data['check_point']
        status[check_point_idx] += 1
        
        if status[check_point_idx] == assist_count:
            pass
        
        print('handler:', assist_idx, status[check_point_idx], message['data'])

In [73]:
p = ar.pubsub()

await p.psubscribe(**{'hq_assist_*_snapshotting': message_handler})

while status[-1] != assist_count:
    
    message = await p.get_message()
        
    if np.sum(status) >= assist_count:
        break

await p.punsubscribe('hq_assist_*_snapshotting')
p.close()

handler: 5 1 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'
handler: 4 2 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'
handler: 2 3 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'
handler: 0 4 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'
handler: 1 5 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'
handler: 6 6 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'
handler: 3 7 b'{"status": "successful", "idx": 3, "check_point": "2021-06-18 09:07:00"}'


In [74]:
message

In [79]:
status

array([0, 0, 0, ..., 0, 0, 0])